In [1]:
import pandas as pd

In [47]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
test_ids = test['PassengerId']

train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [48]:
def clean(data):
    data = data.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], 1)
    
    cols = ['SibSp', 'Parch', 'Fare', 'Age']
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    
    data.Embarked.fillna('U', inplace=True)
    return data

train = clean(train)
test = clean(test)

C:\Users\Hayek\AppData\Local\Temp\ipykernel_11364\3711772868.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], 1)
C:\Users\Hayek\AppData\Local\Temp\ipykernel_11364\3711772868.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], 1)


In [49]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,28.0,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [50]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ['Sex', 'Embarked', 'SibSp', 'Pclass']

for col in cols:
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    print(le.classes_)

train.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']
[0 1 2 3 4 5 8]
[1 2 3]


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,2,1,22.0,1,0,7.2500,2
1,1,0,0,38.0,1,0,71.2833,0
2,1,2,0,26.0,0,0,7.9250,2
3,1,0,0,35.0,1,0,53.1000,2
4,0,2,1,35.0,0,0,8.0500,2


In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train['Survived']
X = train.drop('Survived', axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=26)

In [55]:
clf = LogisticRegression(random_state=0, max_iter=2000).fit(X_train, y_train)

In [56]:
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8432835820895522

In [57]:
submission_preds = clf.predict(test)

In [58]:
df = pd.DataFrame({'PassengerId':test_ids.values,
                   'Survived': submission_preds,
                    })

In [59]:
df.to_csv('submission.csv', index=False)